In [1]:
#ОБРУЧАЛКА

import pandas as pd

# Загужаем файл остатков и отметает лишнее

ost_obr = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
               usecols=["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","Проект"])

ost_obr = ost_obr[ost_obr.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ'])].sort_values('КодСклада')

del ost_obr['Проект']

# Делим файл на две части:магазы и центральные склады

ost_obr1 = ost_obr[ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[~ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[['КодСклада','ТоварНо']]
ost_obr1 = ost_obr1[['СерийныйНомер','ТоварНо']]

# Обработка заборников верхние пустые строки в загружаемом файле должны быть удалены

zab = pd.read_excel(r'C:\анаконда\zab.xlsx')
zab = zab[["Серийный Но.","Товар Но.","Склад-приемник","Номер ЗЛ"]].sort_values(by="Номер ЗЛ",ascending = False)
zab.columns = ['СерийныйНомер','ТоварНо','КодСклада','ЗЛ']
zab.rename(
    columns={"Серийный Но.":'СерийныйНомер', "Товар Но.":'ТоварНо', 
             "Склад-приемник":'КодСклада',"Номер ЗЛ":'ЗЛ'}, 
    inplace=True
) 

# Делим файл на две части:необиркованные и без СН на машине

zab_0 = zab[zab.СерийныйНомер.isnull()]
zab_0 = zab_0[['КодСклада','ТоварНо']]
zab = zab[zab.СерийныйНомер.str.contains('СН',na=False)]
zab = zab[['СерийныйНомер','КодСклада']]

# Проставляем магаз в серийники, которые на 4001/3075/опте

df = pd.merge(ost_obr1,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
ost_obr1 = df[['КодСклада','ТоварНо']]

# Соединяем остатки на магазах,на машине, то что числилось на 4001/3075/опте
result = pd.concat([ost_obr,ost_obr1,zab_0])

# Догружаем размер/артикул/толщ.плетения и тд

df = pd.read_excel(r'C:\анаконда\карточки.xlsx',sheet_name='обручи',header=6)
result = pd.merge(result,df,left_on=['ТоварНо'],right_on=['Номер'],how='left')
del result['Номер']
result.set_index('КодСклада',drop=True, inplace=True)

# Сохраняем в эксель. нужно доработать сохранение в уже существующий эксель

with pd.ExcelWriter(r'C:\Остатки\обручи.xlsx') as writer:
    result.to_excel(writer, sheet_name='остатки')

In [11]:
# ДЕТСКОЕ
import pandas as pd

# Загужаем файлы остатков и заборников

ost = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
               usecols=["КодСклада", "ТоварНо","СерийныйНомер","ТоварноеНаправление"])

zab = pd.read_excel(r'C:\анаконда\zab.xlsx',
                  usecols=["Серийный Но.","Товар Но.","Склад-приемник","Товарное направление"])

# Фильтруем по направлению, в заборниках оставляем серийники, которых нет в остатках

ost = ost[ost['ТоварноеНаправление'].isin(['БК','ИФ'])] 
zab = zab[zab['Товарное направление'].isin(['БК','ИФ']) & ~zab['Серийный Но.'].isin(ost['СерийныйНомер'].tolist())]

# подгоняем колонки, соединяем датафреймы и сохраняем
zab = zab[['Склад-приемник', 'Товар Но.', 'Серийный Но.','Товарное направление']]
zab.columns = ost.columns

final = pd.concat([ost,zab],ignore_index=True, sort=False)
final.to_excel(r'C:\Остатки\остатки.xlsx', index=False)


In [20]:
#БУКВЫ И ЗОДИАКИ

import pandas as pd

# Загужаем файл остатков и отметает лишнее

ost_obr = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
               usecols=["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","ТоварноеНаправление","Проект","Тип1"])

# Оставляем только декоративные подвесы и нужные проекты

ost_obr = ost_obr[(ost_obr["Тип1"] == 'ПОДВЕС ДЕКОРАТИВНЫЙ')]
ost_obr = ost_obr[ost_obr.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ'])].sort_values('КодСклада')
del ost_obr['Проект'], ost_obr['Тип1']

# Делим файл на две части:магазы и центральные склады

ost_obr1 = ost_obr[ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[~ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[['КодСклада','ТоварНо','ТоварноеНаправление']]
ost_obr1 = ost_obr1[['СерийныйНомер','ТоварНо','ТоварноеНаправление']]

# Обработка заборников верхние пустые строки в загружаемом файле должны быть удалены

zab = pd.read_excel(r'C:\анаконда\zab.xlsx',
                  usecols=["Серийный Но.","Товар Но.","Склад-приемник","Номер ЗЛ","Товарное направление","Тип 1"])

zab = zab[(zab["Тип 1"] == 'ПОДВЕС ДЕКОРАТИВНЫЙ')]
del zab["Тип 1"]
zab = zab.sort_values(by="Номер ЗЛ",ascending = False)

zab.rename(
    columns={"Серийный Но.":'СерийныйНомер', "Товар Но.":'ТоварНо', 
             "Склад-приемник":'КодСклада',"Номер ЗЛ":'ЗЛ',"Товарное направление":'ТоварноеНаправление' }, 
    inplace=True
) 

# Делим файл на две части:необиркованные и без СН на машине

zab_0 = zab[zab.СерийныйНомер.isnull()]
zab_0 = zab_0[['КодСклада','ТоварНо','ТоварноеНаправление']]
zab = zab[zab.СерийныйНомер.str.contains('СН',na=False)]
zab = zab[['СерийныйНомер','КодСклада']]

# Проставляем магаз в серийники, которые на 4001/3075/опте

df = pd.merge(ost_obr1,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
ost_obr1 = df[['КодСклада','ТоварНо','ТоварноеНаправление']]

# Соединяем остатки на магазах,на машине, то что числилось на 4001/3075/опте
result = pd.concat([ost_obr,ost_obr1,zab_0])


# Догружаем размер/артикул/толщ.плетения и тд

df = pd.read_excel(r'C:\анаконда\карточки.xlsx',sheet_name='буки',header=6)
result = pd.merge(result,df,left_on=['ТоварНо'],right_on=['Номер'],how='left')
del result['Номер']
result.set_index('КодСклада',drop=True, inplace=True)

# Очищаем описание 3 от ненужного мусора
result['Описание 3'] = result['Описание 3'].apply(lambda a: a[:str(a).find(' ')] if str(a).find(' ')>0 else a )


# Сохраняем в эксель. нужно доработать сохранение в уже существующий эксель

with pd.ExcelWriter(r'C:\Остатки\буквыизодиаки.xlsx') as writer:
    result.to_excel(writer)
    

In [12]:
# Тип1 артикул размер
import pandas as pd
#загужаем файл остатков и отметает лишнее
ost_obr = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
                    usecols=["КодСклада", "НазваниеСклада", "Тип1","ПоставщикАрт",
                                "РазмерИзделия","ТоварНо","СерийныйНомер","Проект"])
ost_obr = ost_obr[ost_obr.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ'])].sort_values('КодСклада')
del ost_obr['Проект']

#делим файл на две части:магазы и центральные склады
ost_obr1 = ost_obr[ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[~ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
del ost_obr1['КодСклада']

#обработка заборников верхние пустые строки в загружаемом файле должны быть удалены
zab = pd.read_excel(r'C:\анаконда\zab.xlsx')
zab = zab[["Серийный Но.","Склад-приемник","Номер ЗЛ"]].sort_values(by="Номер ЗЛ",ascending = False)
zab.columns = ['СерийныйНомер','КодСклада','ЗЛ']
zab.rename(columns={"Серийный Но.":'СерийныйНомер', "Склад-приемник":'КодСклада',"Номер ЗЛ":'ЗЛ'}, inplace=True) 
del zab['ЗЛ']

#проставляем магаз в серийники, которые на 4001/3075/опте
ost_obr1 = pd.merge(ost_obr1,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
ost_obr1 = ost_obr1[["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","Тип1","ПоставщикАрт","РазмерИзделия"]]

#соединяем остатки на магазах,на машине, то что числилось на 4001/3075/опте
result = pd.concat([ost_obr,ost_obr1])
result.set_index('КодСклада',drop=True, inplace=True)
del result['НазваниеСклада']

#сохраняем в эксель. нужно доработать сохранение в уже существующий эксель
with pd.ExcelWriter(r'C:\Остатки\остатки.xlsx') as writer:
    result.to_excel(writer)

In [19]:
#ЛЮБОЙ ТОВАР
import pandas as pd
# Загружаем файл остатков и отметает лишнее

ost_obr = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
                    usecols=["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","Проект"])

ost_obr = ost_obr[ost_obr.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ'])].sort_values('КодСклада')
del ost_obr['Проект']

# Делим файл на две части:магазы и центральные склады

ost_obr1 = ost_obr[ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[~ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[["КодСклада",  "ТоварНо","СерийныйНомер"]]
del ost_obr1['КодСклада']

# обработка заборников верхние пустые строки в загружаемом файле должны быть удалены

zab = pd.read_excel(r'C:\анаконда\zab.xlsx')
zab = zab[["Серийный Но.","Склад-приемник","Номер ЗЛ"]].sort_values(by="Номер ЗЛ",ascending = False)
zab.rename(columns={"Серийный Но.":'СерийныйНомер', "Склад-приемник":'КодСклада',"Номер ЗЛ":'ЗЛ'}, inplace=True) 
del zab['ЗЛ']

# проставляем магаз в серийники, которые на 4001/3075/опте

ost_obr1 = pd.merge(ost_obr1,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
ost_obr1 = ost_obr1[["КодСклада",  "ТоварНо","СерийныйНомер"]]

# соединяем остатки на магазах,на машине, то что числилось на 4001/3075/опте

result = pd.concat([ost_obr,ost_obr1])
result.set_index('КодСклада',drop=True, inplace=True)

# сохраняем в эксель. нужно доработать сохранение в уже существующий эксель

with pd.ExcelWriter(r'C:\Остатки\остатки.xlsx') as writer:
    result.to_excel(writer)

In [70]:

wordDict = {'V' : 'I', 'N' : 'A', 'G' : 'T', 'U' : 'H', 'R' : 'E', 'Z' : 'M', 'F' : 'S', 'J' : 'W', 'O' : 'B', 'Y' : 'L', 'Q' : 'D', 'X' : 'K', 'P' : 'C', 'v' : 'i', 'n' : 'a', 'g' : 't', 'u' : 'h', 'r' : 'e', 'z' : 'm', 'f' : 's', 'j' : 'w', 'o' : 'b', 'y' : 'l', 'q' : 'd', 'x' : 'k', 'p' : 'c', 'I' : 'V', 'A' : 'N', 'T' : 'G', 'H' : 'U', 'E' : 'R', 'M' : 'Z', 'S' : 'F', 'W' : 'J', 'B' : 'O', 'L' : 'Y', 'D' : 'Q', 'K' : 'X', 'C' : 'P', 'i' : 'v', 'a' : 'n', 't' : 'g', 'h' : 'u', 'e' : 'r', 'm' : 'z', 's' : 'f', 'w' : 'j', 'b' : 'o', 'l' : 'y', 'd' : 'q', 'k' : 'x', 'c' : 'p'}

In [12]:

df = pd.DataFrame(data={'col1':["Астрахань", "Барнаул", "Владикавказ", "Вологда", "Грозный", "Екатеринбург", "Иваново", "Ижевск", "Иркутск", "Казань", "Кемерово", "Курган", "Майкоп", "Махачкала", "Москва", "Нальчик", "Нижний Новгород", "Пермь", "Ростов-на-Дону", "Самара", "Санкт-Петербург", "Саратов", "Тверь", "Томск", "Тюмень", "Ульяновск", "Уфа", "Челябинск", "Черкесск", "Ярославль"]})
df.to_excel(r'C:\Остатки\мечети.xlsx')


In [23]:
def multipleReplace(text, wordDict):
    """
    take a text and replace words that match the key in a dictionary
    with the associated value, return the changed text
    """
    for x in text:
        text = text.replace(key, wordDict[key])
    return text

In [71]:
text = 'Vg jnf gur Juvgr Enoovg'
for x in text:
    text = text.replace(x, wordDict[x])
return text
    


KeyError: ' '

In [ ]:
import random
while True:
    first_num = random.randint(2, 11)
    second_num = random.randint(2, 10)
    answer = int(input("Сколько будет {} умножить на {} ".format(first_num,second_num)))
    if answer == first_num*second_num:
        print("Правильно")
    else:
        answer = int(input("Подумай хорошенько "))
        while answer != first_num*second_num:
            answer = int(input("Подумай хорошенько "))

Сколько будет 5 умножить на 2 10
Правильно
Сколько будет 7 умножить на 5 5
Подумай хорошенько 5
Подумай хорошенько 35


In [17]:
zab = pd.read_excel(r'C:\анаконда\zab.xlsx',
                  usecols=["Серийный Но.","Товар Но.","Склад-приемник","Номер ЗЛ","Товарное направление","Тип 1"])


In [27]:

new_zab = zab.groupby(['Тип 1','Товар Но.','Склад-приемник',],as_index=False)[['Серийный Но.']].count()

In [11]:
import pandas as pd

ost1 = pd.read_csv(r'C:\Users\Dotsenko.Semen\Documents\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
               usecols=[2,3,9,10])


In [5]:
ost1.head()

,ТоварНо,СерийныйНомер,ПоставщикАрт
0,ТОВ0832677,СН0044506780,КО 03-00
1,ТОВ0793676,СН0044074026,414
2,ТОВ0981348,СН0036215133,A480400.12ZN#РЫБЫ
3,ТОВ0886702,СН0050856399,С1104960
4,ТОВ0612035,СН0012358845,35350595


In [12]:
sales1 = pd.read_excel(r'C:\анаконда\сер.xlsx')

barlist = sales1.сер.tolist()
ost1['признак'] = ost1.СерийныйНомер.apply(lambda x: x in barlist)

In [7]:
ost1.head()

,ТоварНо,СерийныйНомер,ПоставщикАрт,признак
0,ТОВ0832677,СН0044506780,КО 03-00,False
1,ТОВ0793676,СН0044074026,414,False
2,ТОВ0981348,СН0036215133,A480400.12ZN#РЫБЫ,False
3,ТОВ0886702,СН0050856399,С1104960,False
4,ТОВ0612035,СН0012358845,35350595,False


In [13]:
ost1[ost1.признак == True].to_excel(r'C:\Остатки\ифсер.xlsx',index=False)

In [5]:
import numpy as np

len(np.arange(16.5, 21, 0.5))

9

In [6]:
np.arange(16.5, 21, 0.5)

array([16.5, 17. , 17.5, 18. , 18.5, 19. , 19.5, 20. , 20.5])